### Problem Statement

A company builds computers. A computer is a fairly complex product, and there are several factories that assemble them which the company pays a certain amount per unit. The cost of this computer model on the market is fixed at 500, different factories assemble the computers at different speeds and costs. Factory f0 produces 2000 per day at 450 per unit, factory f1 1500 per day at 420 per unit and f2 1000 per day at 400 per unit. We have 1 month to assemble 80 000 units under the constraint that no factory is to produce more than double the units than any other factory. The question is, what is the optimal production allocation between the factories such that we maximize the profit obtained from selling the computers under those constraints

In [1]:
import numpy as np
import pandas as pd
!pip install pulp
from pulp import *

#### To solve the computer production problem with linear programming, we need the following things:

1. The set of decision variables
2. The set of linear constraints on those variables
3. A linear objective function to maximize or minimize 

In [2]:
problem = LpProblem("problemName", LpMaximize)

We need to create a problem object with the above code:
We are going to add the constraints and objective function to this object.Problem constructor receives a problem name and LpMaximize, which means we want to maximize our objective function. In our case, this is the profit from selling a certain number of computers.

In [3]:
# factory cost per day
cf0 = 450
cf1 = 420
cf2 = 400

# factory throughput per day
f0 = 2000
f1 = 1500
f2 = 1000

# production goal
goal = 80000

# time limit
max_num_days = 30

num_factories = 3

### Decision Variables
In PuLP, a decision variable is defined the following way


In [4]:
# varriable=LpVariable("varriableName")
# var = LpVariable("boundedVariableName",lowerBound,upperBound)
# varDict = LpVariable.dicts("varDict", variableNames, lowBound, upBound)

In [5]:
# factories
num_factories = 3
factory_days = LpVariable.dicts("factoryDays", 
                                list(range(num_factories)), 
                                0, 
                                30, 
                                cat="Continuous")

### Constraints
Now that we defined our decision variables we can shift to defining the constraints for the problem. Note that the constraints have to be linear in a linear programming setting. The constraints that we care about are that the number of units assembled should be above or equal to the goal amount and the production constraint that no factory should produce more than double as the other factory:

In [6]:
# goal constraint
c1 = factory_days[0]*f0 + factory_days[1]*f1 + factory_days[2] * f2 >= goal

# production constraints
c2 = factory_days[0]*f0 <= 2*factory_days[1]*f1
c3 = factory_days[0]*f0 <= 2*factory_days[2]*f2
c4 = factory_days[1]*f1 <= 2*factory_days[2]*f2
c5 = factory_days[1]*f1 <= 2*factory_days[0]*f0
c6 = factory_days[2]*f2 <= 2*factory_days[1]*f1
c7 = factory_days[2]*f2 <= 2*factory_days[0]*f0

# adding the constraints to the problem
problem += c1
problem += c2
problem += c3
problem += c4
problem += c5
problem += c6
problem += c7

In [7]:
# objective function
objCons = -factory_days[0]*cf0*f0 - factory_days[1]*cf1*f1 - factory_days[2]*cf2*f2
problem+=objCons

In [8]:
print(problem)

problemName:
MAXIMIZE
-900000*factoryDays_0 + -630000*factoryDays_1 + -400000*factoryDays_2 + 0
SUBJECT TO
_C1: 2000 factoryDays_0 + 1500 factoryDays_1 + 1000 factoryDays_2 >= 80000

_C2: 2000 factoryDays_0 - 3000 factoryDays_1 <= 0

_C3: 2000 factoryDays_0 - 2000 factoryDays_2 <= 0

_C4: 1500 factoryDays_1 - 2000 factoryDays_2 <= 0

_C5: - 4000 factoryDays_0 + 1500 factoryDays_1 <= 0

_C6: - 3000 factoryDays_1 + 1000 factoryDays_2 <= 0

_C7: - 4000 factoryDays_0 + 1000 factoryDays_2 <= 0

VARIABLES
factoryDays_0 <= 30 Continuous
factoryDays_1 <= 30 Continuous
factoryDays_2 <= 30 Continuous



# Solving 
solve method which outputs 1 if the problem is solved and -1 if it was infeasible.

In [9]:
# solving
problem.solve()

1

In [10]:
for i in range(3):
 print(f"Factory {i}: {factory_days[i].varValue}")

Factory 0: 8.3333333
Factory 1: 22.222222
Factory 2: 30.0


In [11]:
print(f"Factory 0: {factory_days[0].varValue*f0}")
print(f"Factory 1: {factory_days[1].varValue*f1}")
print(f"Factory 2: {factory_days[2].varValue*f2}")

Factory 0: 16666.6666
Factory 1: 33333.333
Factory 2: 30000.0
